In [1]:
import numpy as np

def display_bb(bb: int) -> None:
    board_string = ""
    for i in range(8):
        for j in range(8):
            board_string += "|"
            if ((1 << (63 - (8 * i + j))) & bb) != 0:
                board_string += "X"
            else:
                board_string += " "
        board_string += "|\n"
    print(hex(bb))
    print(board_string)
    # return board_string



In [2]:
display_bb(0xEFEFEFEFEFEFEFEF);
display_bb(0x7F7F7F7F7F7F7F7F);
display_bb(0x3B3B3B3B3B3B3B3B);
display_bb(0x00FF00FF00FF00FF);
display_bb(0x00FF00FF00FF00FF);
display_bb(0x00FF00FF00FF00FF);

display_bb(0xff00);

display_bb((0xff00 << 7) ^ 0xFEFEFEFEFEFEFEF)

display_bb(0x007e7e7e7e7e7e00)

0xefefefefefefefef
|X|X|X| |X|X|X|X|
|X|X|X| |X|X|X|X|
|X|X|X| |X|X|X|X|
|X|X|X| |X|X|X|X|
|X|X|X| |X|X|X|X|
|X|X|X| |X|X|X|X|
|X|X|X| |X|X|X|X|
|X|X|X| |X|X|X|X|

0x7f7f7f7f7f7f7f7f
| |X|X|X|X|X|X|X|
| |X|X|X|X|X|X|X|
| |X|X|X|X|X|X|X|
| |X|X|X|X|X|X|X|
| |X|X|X|X|X|X|X|
| |X|X|X|X|X|X|X|
| |X|X|X|X|X|X|X|
| |X|X|X|X|X|X|X|

0x3b3b3b3b3b3b3b3b
| | |X|X|X| |X|X|
| | |X|X|X| |X|X|
| | |X|X|X| |X|X|
| | |X|X|X| |X|X|
| | |X|X|X| |X|X|
| | |X|X|X| |X|X|
| | |X|X|X| |X|X|
| | |X|X|X| |X|X|

0xff00ff00ff00ff
| | | | | | | | |
|X|X|X|X|X|X|X|X|
| | | | | | | | |
|X|X|X|X|X|X|X|X|
| | | | | | | | |
|X|X|X|X|X|X|X|X|
| | | | | | | | |
|X|X|X|X|X|X|X|X|

0xff00ff00ff00ff
| | | | | | | | |
|X|X|X|X|X|X|X|X|
| | | | | | | | |
|X|X|X|X|X|X|X|X|
| | | | | | | | |
|X|X|X|X|X|X|X|X|
| | | | | | | | |
|X|X|X|X|X|X|X|X|

0xff00ff00ff00ff
| | | | | | | | |
|X|X|X|X|X|X|X|X|
| | | | | | | | |
|X|X|X|X|X|X|X|X|
| | | | | | | | |
|X|X|X|X|X|X|X|X|
| | | | | | | | |
|X|X|X|X|X|X|X|X|

0xff00
| | | | | | | |

In [3]:
pawns = 0x1028c700

display_bb(pawns)

pawnAttacks   = (pawns   << 7) & 0xFEFEFEFEFEFEFEF;
centerPawns = pawns & 0x7e7e7e7e7e7e00;
leftPawns   = pawns & 0x80808080808000;
rightPawns  = pawns & 0x01010101010100;

pawnAttacks = ((centerPawns << 9) | (centerPawns << 7) | (leftPawns << 7) | (rightPawns << 9));
display_bb(pawnAttacks)

0x1028c700
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | |X| | | | |
| | |X| |X| | | |
|X|X| | | |X|X|X|
| | | | | | | | |

0x2854ef0000
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | |X| |X| | | |
| |X| |X| |X| | |
|X|X|X| |X|X|X|X|
| | | | | | | | |
| | | | | | | | |



In [4]:
display_bb(pawns | pawnAttacks)

0x2854efc700
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | |X| |X| | | |
| |X| |X| |X| | |
|X|X|X| |X|X|X|X|
|X|X| | | |X|X|X|
| | | | | | | | |



In [5]:
bishops = 0x020200000000
display_bb(bishops)

0x20200000000
| | | | | | | | |
| | | | | | | | |
| | | | | | |X| |
| | | | | | |X| |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |



In [ ]:

def propagateDirection(b:int, friendly:int, enemy:int, direction:int, limit = False)-> int:
    '''
    Direction guide:
    
    |7|6|5|
    |4| |3|
    |2|1|0|

    '''
    if b == 0:
        return 0
    # base masks
    dwnCap   = 0xff
    upCap    = dwnCap << 56
    rightCap = 0x0101010101010101
    leftCap  = rightCap << 7

    # For directions 0..7, set a bit if that direction includes the component
    DOWN_BITS  = 0b00000111  # dirs {0,1,2}
    UP_BITS    = 0b11100000  # dirs {5,6,7}
    RIGHT_BITS = 0b00101001  # dirs {0,3,5}
    LEFT_BITS  = 0b10010100  # dirs {2,4,7}


    d = direction & 7  # ensure 0..7
    cap  = ((DOWN_BITS  >> d) & 1) * dwnCap
    cap |= ((UP_BITS    >> d) & 1) * upCap
    cap |= ((RIGHT_BITS >> d) & 1) * rightCap
    cap |= ((LEFT_BITS  >> d) & 1) * leftCap


    safe_b = b & ~(cap|enemy) #and with complement to remove boundary pieces and captures propagating further

    

    shiftUp = direction > 3
    levelShift = (direction != 3) and (direction !=4) 
    shiftAmt = 1
    if levelShift:
        shiftAmt = direction + 2 if shiftUp else 9 - direction
    
    next = safe_b << shiftAmt if shiftUp else safe_b >> shiftAmt
    
    next = next & ~friendly


    if  limit:
        return next
    else:
        return next | propagateDirection(next, friendly, enemy, direction)


In [82]:
def propagateBishop(b:int, friendly:int = 0, enemy:int = 0, limit = False)-> int:
    return propagateDirection(b, friendly, enemy, 7, limit) | \
        propagateDirection(b, friendly, enemy, 5, limit) | \
        propagateDirection(b, friendly, enemy, 2, limit) | \
        propagateDirection(b, friendly, enemy, 0, limit)

def propagateRook(b:int, friendly:int = 0, enemy:int = 0, limit=False)-> int:
    return propagateDirection(b, friendly, enemy, 6, limit) | \
        propagateDirection(b, friendly, enemy, 4, limit) | \
        propagateDirection(b, friendly, enemy, 3, limit) | \
        propagateDirection(b, friendly, enemy, 1, limit)

def propagateQueen(b:int, friendly:int = 0, enemy:int = 0, limit=False)-> int:
    return propagateRook(b, friendly, enemy, limit) | propagateBishop(b, friendly, enemy, limit)

def propagateKing(b:int, friendly:int = 0, enemy:int = 0)-> int:
    return propagateQueen(b, friendly, enemy, limit=True)



0x10000000000000000
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |



In [ ]:

def propagateKnight(b:int, friendly:int = 0, enemy:int = 0)-> int:
    
    '''
    Knight movement guide
    | |7| |6| |
    |5| | | |4|
    | | |X| | |
    |3| | | |2|
    | |1| |0| |

    Shift amounts
    0: >> 17
    1: >> 15
    2: >> 10
    3: >> 6
    4: << 6
    5: << 10
    6: << 15
    7: << 17

    '''
    
    
    full_board = 0xffffffffffffffff
    # base masks
    dwnCap   = 0xff
    upCap    = dwnCap << 56
    rightCap = 0x0101010101010101
    leftCap  = rightCap << 7

    twoDwnCap = dwnCap << 8
    twoUpCap  = upCap >> 8
    twoRightCap = rightCap << 1
    twoLeftCap = leftCap >> 1 

    

    # For directions 0..7, set a bit if that direction includes the component
    DOWN_BITS      = 0b00001111  # dirs {0,1,2,3}
    UP_BITS        = 0b11110000  # dirs {4,5,6,7}
    RIGHT_BITS     = 0b01010101  # dirs {0,2,4,6}
    LEFT_BITS      = 0b10101010  # dirs {1,3,5,7}

    TWO_DOWN_BITS  = 0b00000011  # dirs {0,1}
    TWO_UP_BITS    = 0b11000000  # dirs {6,7}
    TWO_RIGHT_BITS = 0b00010100  # dirs {2,4}
    TWO_LEFT_BITS  = 0b00101000  # dirs {3,5}
    
    shiftAmts = [-17, -15, -10, -6, 6, 10, 15, 17]
    next = 0
    # for s in shiftAmts:
    for d in range(8):
        cap  = ((DOWN_BITS      >> d) & 1) * dwnCap
        cap |= ((UP_BITS        >> d) & 1) * upCap
        cap |= ((RIGHT_BITS     >> d) & 1) * rightCap
        cap |= ((LEFT_BITS      >> d) & 1) * leftCap
        cap |= ((TWO_DOWN_BITS  >> d) & 1) * twoDwnCap
        cap |= ((TWO_UP_BITS    >> d) & 1) * twoUpCap
        cap |= ((TWO_RIGHT_BITS >> d) & 1) * twoRightCap
        cap |= ((TWO_LEFT_BITS  >> d) & 1) * twoLeftCap
        safe_b = b & ~(cap)
        # display_bb(cap)
        # display_bb(safe_b)
        # display_bb(safe_b)
        s = shiftAmts[d]
        next |= safe_b >> -s if s < 0 else safe_b << s
    next &= full_board
    next &= ~friendly
    return next
    

    

In [ ]:
bishops = 0x8000000000
display_bb(bishops)
# display_bb(friendly)
# display_bb(enemy)
display_bb(propagateKnight(bishops))
# display_bb(propagateDirection(bishops, friendly=0, enemy = 0, direction = 2))

#3 4

0x8000000000
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
|X| | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |

0x200020400000
| | | | | | | | |
| | | | | | | | |
| | |X| | | | | |
| | | | | | | | |
| | |X| | | | | |
| |X| | | | | | |
| | | | | | | | |
| | | | | | | | |



In [85]:
display_bb(upCap)

0xff00000000000000
|X|X|X|X|X|X|X|X|
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |



In [86]:
bin(0b0011 ^ 0b0101)

'0b110'

In [89]:
bishops = 0x8080000000
display_bb(bishops)
# display_bb(friendly)
# display_bb(enemy)
display_bb(propagateBishop(bishops,friendly, enemy))
# display_bb(propagateDirection(bishops, friendly=0, enemy = 0, direction = 2))

#3 4

0x8080000000
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
|X| | | | | | | |
|X| | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |

0x30604040000000
| | | | | | | | |
| | |X|X| | | | |
| |X|X| | | | | |
| |X| | | | | | |
| |X| | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |



In [78]:
enemy = 0xff << 48
display_bb(enemy)

friendly = 0xff0000
display_bb(friendly)

0xff000000000000
| | | | | | | | |
|X|X|X|X|X|X|X|X|
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |

0xff0000
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
|X|X|X|X|X|X|X|X|
| | | | | | | | |
| | | | | | | | |



In [79]:
queens = 0x4000000
display_bb(propagateQueen(queens,friendly, enemy))

6
0x4000000
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | |X| | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |

6
0x400000000
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | |X| | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |

6
0x40000000000
| | | | | | | | |
| | | | | | | | |
| | | | | |X| | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |

6
0x0
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |

4
0x4000000
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | |X| | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |

4
0x8000000
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | |X| | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |

4
0x10000000
| | | | | | | | |
| | | | | | | | |
| | | | |

In [80]:
display_bb(propagateKing(bishops,friendly, enemy))

6
0x8080000000
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
|X| | | | | | | |
|X| | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |

4
0x0
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |

3
0x8080000000
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
|X| | | | | | | |
|X| | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |

1
0x8080000000
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
|X| | | | | | | |
|X| | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |

7
0x0
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |

5
0x8080000000
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
|X| | | | | | | |
|X| | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |

2
0x0
| | | | | | | | |
| | | | | | | | |
| | | | | | | | 